<a href="https://colab.research.google.com/github/hugeclear/image_recognition/blob/master/app_recognitioin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit==1.7.0 --quiet
!pip install pyngrok==4.1.1 --quiet

In [3]:
import streamlit as st
from pyngrok import ngrok
import numpy as np
import json
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torchvision
from torchvision import models,transforms


In [4]:
print("pytorch version: ",torch.__version__)
print("torchvision: " , torchvision.__version__)


pytorch version:  1.12.0+cu113
torchvision:  0.13.0+cu113


In [6]:
import os
import urllib.request
import zipfile
data_dir = "./data/"
if not os.path.exists(data_dir):
  os.mkdir(data_dir)
#imagenetのclass_index download 
# made by keras
# https://github.com/fchollet/deep-learning-models/blob/master/imagenet_utils.py
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
save_path = os.path.join(data_dir,"imagenet_class_index.json")

if not os.path.exists(save_path):
  urllib.request.urlretrieve(url,save_path)

url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
save_path = os.path.join(data_dir,"hymenoptera_data.zip")

if not os.path.exists(save_path):
  urllib.request.urlretrieve(url,save_path)

  zip = zipfile.ZipFile(save_path)
  zip.extractall(data_dir) #unfreeze
  zip.close()

In [50]:
%%writefile model.py
import torch
from torchvision import models, transforms
import torchvision
import numpy as np
import json


use_pretrained=True
net=models.vgg16(pretrained=use_pretrained) #import pretraiined model
net.eval()

def predict(img):
  #前処理
  transform = transforms.Compose([
      transforms.Resize(224),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize(
          mean=[0.485,0.456,0.406],
          std=[0.229,0.224,0.225]
          )
  ])
  #input to model
  img_transformed = transform(img)
  #img_transformed = img_transformed.numpy().transpose((1,2,0))
  #img_transformed = np.clip(img_transformed,0,1) 
  inputs=img_transformed.unsqueeze_(0) #unsqueeze_() ->inplace tensor , unsqueeze() -> not inplace

  #Create a post-processing class to predict labels from output results
  ILSVRC_class_index = json.load(open('/content/data/imagenet_class_index/json','r'))


  net.eval()
  out=net(inputs)
  maxid = np.argmax(out.detach().numpy())
  predicted_label_name = ILSVRC_class_index[str(maxid)][1]  

  result = predicted_label_name
  return result
  

   
  
  


Overwriting model.py


In [26]:
%%writefile app.py
import streamlit as st
import matplotlib.pyplot as plt
from PIL import Image
from model import predict

st.set_option("deprecation.showfileUploaderEncoding",False)
st.sidebar.title("画像認識アプリ")
st.sidebar.write("vgg１６で未知の画像を推定")

st.sidebar.write("")

img_source=st.sidebar.radio("select image source",
                            ("upload image","take photo by camera"))
if img_source == "upload image":
  img_file = st.sidebar.file_uploader("select image",type=["png","jpg"])
elif img_source=="take photo by camera":
  img_file = st.camera_input("take photo")

if img_file is not None:
  with st.spinner("estimating..."):
    img = Image.open(img_file)
    st.image(img, caption="this image is ......")
  
    results = predict(img)
    st.write(results+"です")

    st.suheader("results")
    st.write()


Overwriting app.py


In [12]:
!ngrok authtoken 299DaPckUttczCFfo6GrbOSuLZK_6neHqb4NvQuBTqnd3xKjm

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [51]:
!streamlit run app.py &>/dev/null&

In [52]:
ngrok.kill()
url = ngrok.connect(port="8501")


INFO:pyngrok.process:Killing ngrok process: 1971
2022-08-12 09:29:39.012 INFO    pyngrok.process: Killing ngrok process: 1971
INFO:pyngrok.process:ngrok process starting: 2014
2022-08-12 09:29:39.067 INFO    pyngrok.process: ngrok process starting: 2014
INFO:pyngrok.process:t=2022-08-12T09:29:39+0000 lvl=info msg="no configuration paths supplied"

2022-08-12 09:29:39.100 INFO    pyngrok.process: t=2022-08-12T09:29:39+0000 lvl=info msg="no configuration paths supplied"

INFO:pyngrok.process:t=2022-08-12T09:29:39+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2022-08-12 09:29:39.119 INFO    pyngrok.process: t=2022-08-12T09:29:39+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

INFO:pyngrok.process:t=2022-08-12T09:29:39+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2022-08-12 09:29:39.125 INFO    pyngrok.process: t=2022-08-12T09:29:39+0000 lvl=info msg="open config file" 

In [53]:
print(url)

2022-08-12 09:29:39.293 INFO    pyngrok.process: t=2022-08-12T09:29:39+0000 lvl=info msg=end pg=/api/tunnels id=21f566ac7e7095f0 status=201 dur=54.792034ms



http://0d01-34-85-168-202.ngrok.io
